In [4]:
import av
import cv2
import numpy as np
import pandas as pd

In [5]:
images_av = []
images_cv2 = []


In [20]:
import av

container = av.open('/work/bjuncek/AVA_actions/data/train/_145Aa_xkuE.mp4')
float(container.duration / av.time_base)


2154.418

In [26]:
print(float(container.streams.audio[0].duration * container.streams.audio[0].time_base), float(container.streams.video[0].duration * container.streams.video[0].time_base))

2154.417052154195 2154.36


In [27]:
container.streams

In [17]:
%%timeit
cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

188 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
images_av = []
images_cv2 = []
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())

cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

In [19]:

print(len(images_av), len(images_cv2))

327 327


In [1]:
import torchvision
torchvision.set_video_backend("video_reader")

In [2]:
vframes, _, _ = torchvision.io.read_video("./videos/WUzgd7C1pWA.mp4")
vframes.size()

/Users/bjuncek/bin/conda/envs/video_benchmark/lib/python3.7/site-packages/torchvision-0.8.0a0+9b80465-py3.7-macosx-10.9-x86_64.egg/torchvision/io/_video_opt.py:479: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


torch.Size([327, 256, 340, 3])

# Simple measurement of python overhead

In [1]:
# first, import torch
import torch

In [2]:
%%timeit
# torch multitensor
a = torch.zeros(1000, 3, 112, 112)

70.4 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
# for loop with array allocation
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))

93 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
# for loop with array allocation and stacking
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))
a = torch.stack(test)


179 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exploring DECORD

With GPU and CPU support (or at least I hope so)

In [8]:
import torchvision
torchvision.set_video_backend("video_reader")
vframes, _, _ = torchvision.io.read_video('/work/bjuncek/AVA_actions/data/train/2FIHxnZKg6A.webm')
print("Reference number of frames", vframes.size())

/home/bjuncek/miniconda3/envs/tv08/lib/python3.7/site-packages/torchvision-0.8.0a0+344603-py3.7-linux-x86_64.egg/torchvision/io/_video_opt.py:479: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 104530636800 bytes. Error code 12 (Cannot allocate memory)

In [13]:
from decord import VideoReader
from decord import cpu
import numpy as np
vr = VideoReader('/work/bjuncek/AVA_actions/data/train/2FIHxnZKg6A.webm', ctx=cpu(0))
# a file like object works as well, for in-memory decoding
nf = len(vr)
# 1. the simplest way is to directly access frames
print("Decord VR number of frames", nf)

Decord VR number of frames 122228


In [14]:
frames = []
for i in range(nf):
    # the video reader will handle seeking and skipping in the most efficient manner
    frames.append(vr[i])
print("Decord VR practical number of frames", len(frames))

Decord VR practical number of frames 122228


In [ ]:
frames

In [36]:
frames = vr.get_batch(range(nf))
print("Decord VR-batch practical number of frames", frames.shape[0])

Decord VR-batch practical number of frames 332


## Decord VideoLoader

uris : list of str
    List of video paths.
ctx : decord.Context or list of Context
    The context to decode the video file, can be decord.cpu() or decord.gpu().
    If ctx is a list, videos will be evenly split over many ctxs.
shape : tuple
    Returned shape of the batch images, e.g., (2, 320, 240, 3) as (Batch, H, W, 3)
interval : int
    Intra-batch frame interval.
skip : int
    Inter-batch frame interval.
shuffle : int
    Shuffling strategy. Can be
    `0`:  all sequential, no seeking, following initial filename order
    `1`:  random filename order, no random access for each video, very efficient
    `2`:  random order
    `3`:  random frame access in each video only.

In [44]:
from decord import VideoLoader
from decord import cpu

In [48]:
vr = VideoLoader(['videos/SOX5yA1l24A.mp4'], shape=(50, 256, 340, 3), ctx=cpu(0), interval=0, skip=0, shuffle=1)
print(len(vr))

6


In [50]:
for i in range(len(vr)):
    frames, idx = vr.next()


AttributeError: 'NDArray' object has no attribute 'tolist'

In [53]:
a = [3, 9]
np.mean(a)

6.0

# Testing for API

In [149]:
import av
container = av.open('videos/SOX5yA1l24A.mp4')

In [74]:
print([x.type for x in list(container.streams)])

['video', 'audio']


In [138]:
container.decode(container.streams.video).__next__()
    

EOFError: [Errno 541478725] End of file

In [125]:
for frame in container.decode(container.streams.audio[0]):
    print(frame.pts)
    

In [139]:
container.seek(200000, stream=container.streams.audio[0])

In [141]:
print(container.streams.audio[0].pts)

AttributeError: pts

In [115]:
container.seek(516096, stream=container.streams.video[0])

In [93]:
keyframes = []
for packet in container.demux():
    if packet.is_keyframe:
        keyframes.append(packet.pts)
print(keyframes)

[2048, 3072, 4096, 5120, 6144, 7168, 8192, 9216, 10240, 11264, 12288, 13312, 14336, 15360, 16384, 17408, 18432, 19456, 20480, 21504, 22528, 23552, 24576, 25600, 26624, 27648, 28672, 29696, 30720, 31744, 32768, 33792, 34816, 35840, 36864, 37888, 38912, 39936, 40960, 41984, 43008, 44032, 45056, 46080, 47104, 48128, 49152, 50176, 51200, 52224, 53248, 54272, 55296, 56320, 57344, 58368, 59392, 60416, 61440, 62464, 63488, 64512, 65536, 66560, 67584, 68608, 69632, 70656, 71680, 72704, 73728, 74752, 75776, 76800, 77824, 78848, 79872, 80896, 81920, 82944, 83968, 84992, 86016, 87040, 88064, 89088, 90112, 91136, 92160, 93184, 94208, 95232, 96256, 97280, 98304, 99328, 100352, 101376, 102400, 103424, 104448, 105472, 106496, 107520, 108544, 109568, 110592, 111616, 112640, 113664, 114688, 115712, 116736, 117760, 118784, 119808, 120832, 121856, 122880, 123904, 124928, 125952, 126976, 128000, 129024, 130048, 131072, 132096, 133120, 134144, 135168, 136192, 137216, 138240, 139264, 140288, 141312, 142336,

In [134]:
import math
def _sec_to_stream(ts, stream):
    return int(math.floor(ts * (1 / stream.time_base)))

def _stream_to_sec(pts, stream):
    return float(pts * stream.time_base)

In [136]:
_sec_to_stream(20, container.streams.audio[0])

960000

In [135]:
_stream_to_sec(600000, container.streams.video[0])

20.0

In [152]:
type(list(container.streams)[0])

av.video.stream.VideoStream

In [154]:
isinstance(list(container.streams)[0], av.stream.Stream)

True

In [174]:
container

<av.InputContainer 'videos/SOX5yA1l24A.mp4'>

In [161]:
container.seek(20)
packet = next(container.demux(video=0))
print(packet.pts)

0


In [162]:
container.seek(0)

In [166]:
float(container.streams.video[0].duration * container.streams.video[0].time_base)

11.077733333333333

In [167]:
float(container.streams.audio[0].duration * container.streams.audio[0].time_base)

11.06

## Understanding seek in PYAV

In [204]:
# getting keyframes
container = av.open("./videos/SOX5yA1l24A.mp4")
# Signal that we only want to look at keyframes.
stream = container.streams.video[0]
stream.codec_context.skip_frame = 'NONKEY'
keyframes = []
for frame in container.decode(stream):
    keyframes.append(frame.pts)
print(keyframes)

[0, 138138, 219219, 292292]


In [205]:
# getting all frames
container = av.open("./videos/SOX5yA1l24A.mp4")
# Signal that we only want to look at keyframes.
stream = container.streams.video[0]
all_frames = []
for frame in container.decode(stream):
    all_frames.append(frame.pts)
print(all_frames)

[0, 1001, 2002, 3003, 4004, 5005, 6006, 7007, 8008, 9009, 10010, 11011, 12012, 13013, 14014, 15015, 16016, 17017, 18018, 19019, 20020, 21021, 22022, 23023, 24024, 25025, 26026, 27027, 28028, 29029, 30030, 31031, 32032, 33033, 34034, 35035, 36036, 37037, 38038, 39039, 40040, 41041, 42042, 43043, 44044, 45045, 46046, 47047, 48048, 49049, 50050, 51051, 52052, 53053, 54054, 55055, 56056, 57057, 58058, 59059, 60060, 61061, 62062, 63063, 64064, 65065, 66066, 67067, 68068, 69069, 70070, 71071, 72072, 73073, 74074, 75075, 76076, 77077, 78078, 79079, 80080, 81081, 82082, 83083, 84084, 85085, 86086, 87087, 88088, 89089, 90090, 91091, 92092, 93093, 94094, 95095, 96096, 97097, 98098, 99099, 100100, 101101, 102102, 103103, 104104, 105105, 106106, 107107, 108108, 109109, 110110, 111111, 112112, 113113, 114114, 115115, 116116, 117117, 118118, 119119, 120120, 121121, 122122, 123123, 124124, 125125, 126126, 127127, 128128, 129129, 130130, 131131, 132132, 133133, 134134, 135135, 136136, 137137, 138138, 

In [210]:
container = av.open("./videos/SOX5yA1l24A.mp4")
stream = container.streams.video[0]
all_pairs = []
for i in range(0, 400000, 1000):
    container.seek(i, backward=True, any_frame=True, stream=stream)
    for frame in container.decode(stream):
        all_pairs.append((i, frame.pts))
        break
print(all_pairs)

[(0, 0), (1000, 0), (2000, 138138), (3000, 138138), (4000, 138138), (5000, 138138), (6000, 138138), (7000, 138138), (8000, 138138), (9000, 138138), (10000, 138138), (11000, 138138), (12000, 138138), (13000, 138138), (14000, 138138), (15000, 138138), (16000, 138138), (17000, 138138), (18000, 138138), (19000, 138138), (20000, 138138), (21000, 138138), (22000, 138138), (23000, 138138), (24000, 138138), (25000, 138138), (26000, 138138), (27000, 138138), (28000, 138138), (29000, 138138), (30000, 138138), (31000, 138138), (32000, 138138), (33000, 138138), (34000, 138138), (35000, 138138), (36000, 138138), (37000, 138138), (38000, 138138), (39000, 138138), (40000, 138138), (41000, 138138), (42000, 138138), (43000, 138138), (44000, 138138), (45000, 138138), (46000, 138138), (47000, 138138), (48000, 138138), (49000, 138138), (50000, 138138), (51000, 138138), (52000, 138138), (53000, 138138), (54000, 138138), (55000, 138138), (56000, 138138), (57000, 138138), (58000, 138138), (59000, 138138), (6

In [211]:
container = av.open("./videos/SOX5yA1l24A.mp4")
stream = container.streams.video[0]
all_pairs = []
for i in range(0, 400000, 1000):
    container.seek(i, backward=True, any_frame=False, stream=stream)
    for frame in container.decode(stream):
        all_pairs.append((i, frame.pts))
        break
print(all_pairs)

[(0, 0), (1000, 0), (2000, 0), (3000, 0), (4000, 0), (5000, 0), (6000, 0), (7000, 0), (8000, 0), (9000, 0), (10000, 0), (11000, 0), (12000, 0), (13000, 0), (14000, 0), (15000, 0), (16000, 0), (17000, 0), (18000, 0), (19000, 0), (20000, 0), (21000, 0), (22000, 0), (23000, 0), (24000, 0), (25000, 0), (26000, 0), (27000, 0), (28000, 0), (29000, 0), (30000, 0), (31000, 0), (32000, 0), (33000, 0), (34000, 0), (35000, 0), (36000, 0), (37000, 0), (38000, 0), (39000, 0), (40000, 0), (41000, 0), (42000, 0), (43000, 0), (44000, 0), (45000, 0), (46000, 0), (47000, 0), (48000, 0), (49000, 0), (50000, 0), (51000, 0), (52000, 0), (53000, 0), (54000, 0), (55000, 0), (56000, 0), (57000, 0), (58000, 0), (59000, 0), (60000, 0), (61000, 0), (62000, 0), (63000, 0), (64000, 0), (65000, 0), (66000, 0), (67000, 0), (68000, 0), (69000, 0), (70000, 0), (71000, 0), (72000, 0), (73000, 0), (74000, 0), (75000, 0), (76000, 0), (77000, 0), (78000, 0), (79000, 0), (80000, 0), (81000, 0), (82000, 0), (83000, 0), (840

In [17]:
# AVA Dataset
import pandas as pd
## seting up and reading it
root_dir = "/work/bjuncek/AVA_actions/ava_val_v2.2.csv"
df = pd.read_csv(root_dir, names=["video_id", "middle_frame_timestamp", "x1", "y1", "x2", "y2", "action_id", "person_id"])
df.head()

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id,person_id
0,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,12,1
1,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,79,1
2,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,12,0
3,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,17,0
4,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,79,0


In [19]:
df = (df.groupby(['video_id', 'middle_frame_timestamp'], as_index=False)
      .agg(lambda x: list(x)))
df

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id,person_id
0,1j20qq1JyX4,902,"[0.002, 0.002, 0.444, 0.444, 0.444]","[0.11800000000000001, 0.11800000000000001, 0.0...","[0.7140000000000001, 0.7140000000000001, 0.992...","[0.977, 0.977, 0.99, 0.99, 0.99]","[12, 79, 12, 17, 79]","[1, 1, 0, 0, 0]"
1,1j20qq1JyX4,903,"[0.02, 0.02, 0.02]","[0.069, 0.069, 0.069]","[0.985, 0.985, 0.985]","[0.985, 0.985, 0.985]","[12, 17, 79]","[2, 2, 2]"
2,1j20qq1JyX4,904,"[0.015, 0.015, 0.015]","[0.052000000000000005, 0.052000000000000005, 0...","[1.0, 1.0, 1.0]","[0.9790000000000001, 0.9790000000000001, 0.979...","[12, 17, 79]","[2, 2, 2]"
3,1j20qq1JyX4,905,"[0.193, 0.193]","[0.016, 0.016]","[1.0, 1.0]","[0.978, 0.978]","[11, 74]","[3, 3]"
4,1j20qq1JyX4,906,"[0.175, 0.175, 0.175]","[0.001, 0.001, 0.001]","[0.995, 0.995, 0.995]","[1.0, 1.0, 1.0]","[11, 74, 80]","[3, 3, 3]"
...,...,...,...,...,...,...,...,...
50247,zC5Fh2tTS1U,1793,"[0.257, 0.257, 0.257, 0.374, 0.374, 0.374]","[0.415, 0.415, 0.415, 0.43799999999999994, 0.4...","[0.401, 0.401, 0.401, 0.514, 0.514, 0.514]","[0.632, 0.632, 0.632, 0.845, 0.845, 0.845]","[12, 79, 80, 12, 79, 80]","[386, 386, 386, 387, 387, 387]"
50248,zC5Fh2tTS1U,1794,"[0.309, 0.309, 0.309, 0.434, 0.434, 0.434]","[0.203, 0.203, 0.203, 0.203, 0.203, 0.203]","[0.455, 0.455, 0.455, 0.586, 0.586, 0.586]","[0.419, 0.419, 0.419, 0.797, 0.797, 0.797]","[12, 74, 80, 12, 79, 80]","[386, 386, 386, 387, 387, 387]"
50249,zC5Fh2tTS1U,1795,"[0.373, 0.373, 0.373, 0.505, 0.505]","[0.122, 0.122, 0.122, 0.14400000000000002, 0.1...","[0.525, 0.525, 0.525, 0.63, 0.63]","[0.344, 0.344, 0.344, 0.42700000000000005, 0.4...","[12, 74, 80, 12, 79]","[386, 386, 386, 387, 387]"
50250,zC5Fh2tTS1U,1796,"[0.515, 0.515, 0.515]","[0.14300000000000002, 0.14300000000000002, 0.1...","[0.639, 0.639, 0.639]","[0.431, 0.431, 0.431]","[12, 79, 80]","[387, 387, 387]"


In [1]:
from torchaudio.io import StreamReader
sr = StreamReader("./videos/SOX5yA1l24A.mp4")

RuntimeError: StreamReader requires FFmpeg extension which is not available. Please refer to the stacktrace above for how to resolve this.